In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
from simpleLSTMWithNNetModel import Model, reloadModel
from trainModel import trainModel
import tensorflow as tf
import os
import pprint
import numpy as np
pp = pprint.PrettyPrinter(indent=4)

In [3]:
# data_train = "data/icd9NotesDataTable_train.csv"
# data_valid = "data/icd9NotesDataTable_valid.csv"
data_train = "data/smallIcd9NotesDataTable_train.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# START_TOKEN = "<s>"
# END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 500
max_grad_norm = 5
codeIdx = 9
textIdx = 6
num_layers = 4
learning_rate = 0.001
training_epochs = 5
batch_size = 2
n_input = 1
# n_steps = 10
n_hidden = 200
# n_classes = helper.n_labels
output_keep_prob = 0.5
input_keep_prob = 1# not sure why there are two but example used 0.5 for output. Check with TA
numLayers = 1
# embeddingSize = embeddings.shape[1]
# print('Embedding size is %d'%(embeddingSize))


In [4]:
output_path = input(prompt = 'Where should models and performances be saved to?')
if output_path == '':
    output_path = 'results/temp'
if not os.path.exists(output_path):
    os.makedirs(output_path)

Where should models and performances be saved to?results/tempDelete


In [5]:
# helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
#     data_train = data_train, data_valid = data_valid, 
#     maxAllowedNoteLength = maxAllowedNoteLength, 
#     codeIdx = codeIdx, textIdx = textIdx,
#     helperLoadPath = 'results/temp/')
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx)
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(output_path)# token2id and max length saved to output_path
# np.savetxt(os.path.join(output_path, 'yDev.gz'),yDev)
# #np.save(os.path.join(modelRunOutputPath, 'xDev.npy'), xDev)
# np.savetxt(os.path.join(output_path, 'devTrueIdxs.gz'), lastTrueWordIdx_dev)

total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch %d'%(total_batches))
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

Total number of batches per epoch 3
Max note length 4
Number of Icd9 codes 6
There are a total of 6 icd9 codes
{'123': 4, '18': 3, '4240': 1, '45': 5, '456': 0, '486': 2}
icd9 present
xDev shape: nObs = 7, nWords = 4
yDev shape: nObs = 7, nClasses = 6
xTrain shape: nObs = 7, nWords = 4
yTrain shape: nObs = 7, nClasses = 6
Embeddings shape: nWords = 13, wordVec len = 50


In [15]:
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

Max note length 4
Number of Icd9 codes 6
There are a total of 6 icd9 codes
{'123': 4, '18': 3, '4240': 1, '45': 5, '456': 0, '486': 2}
icd9 present
xDev shape: nObs = 7, nWords = 4
yDev shape: nObs = 7, nClasses = 6
xTrain shape: nObs = 7, nWords = 4
yTrain shape: nObs = 7, nClasses = 6
Embeddings shape: nWords = 13, wordVec len = 50


In [6]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers}
pp.pprint(hyperParamDict)
# with open(os.path.join(modelRunOutputPath, 'hyperParamDict.pickle'), 'wb') as handle:
#     pickle.dump(hyperParamsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

{   'EMBED_SIZE': 50,
    'batchSize': 2,
    'inputKeepProb': 1,
    'learningRate': 0.001,
    'maxGradNorm': 5,
    'maxNoteLength': 500,
    'n_hidden': 200,
    'numLayers': 1,
    'outputKeepProb': 0.5,
    'trainingEpochsMax': 5}


In [18]:
from trainModel import trainModel
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trainModel(helperObj = helper, embeddings = embeddings, hyperParamDict = hyperParamDict, 
          xDev = xDev, xTrain = xTrain, yDev = yDev, yTrain = yTrain, 
           lastTrueWordIdx_dev = lastTrueWordIdx_dev, 
           lastTrueWordIdx_train = lastTrueWordIdx_train,
           training_epochs = training_epochs, 
           output_path = output_path, batchSizeTrain = batch_size,
           maxIncreasingLossCount = 3, batchSizeDev = 2, chatty = False)

/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***************************
***************************
Running on epoch 0
***************************
***************************
average training loss 0.890275
test loss 0.624925
Total run time was 0.219607
New best model found. Saving
results/temp
***************************
***************************
Running on epoch 1
***************************
***************************
average training loss 0.796055
test loss 0.558762
Total run time was 0.160995
New best model found. Saving
results/temp
***************************
***************************
Running on epoch 2
***************************
***************************
average training loss 0.706315
test loss 0.500679
Total run time was 0.100546
New best model found. Saving
results/temp
***************************
***************************
Running on epoch 3
***************************
***************************
average training loss 0.584277
test loss 0.454022
Total run time was 0.190874
New best model found. Saving
results/t

In [20]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
with tf.Session() as session:
    tf.global_variables_initializer().run()
    modelDict = reloadModel(session = session,
                            saverCheckPointPath = 'results/temp/',
                            saverMetaPath = 'results/temp/bestModel.meta')
    for i in range(3):
        pred_y = session.run(modelDict['y_last'],feed_dict={modelDict['xPlaceHolder']: xDev,
                                      modelDict['trueWordIdxs']:lastTrueWordIdx_dev,
                                      modelDict['outputKeepProb']: 1.0,
                                      modelDict['inputKeepProb']: 1.0}, ) 
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('test loss %f'%(validLoss))
        print('***********************************************')

test loss 0.427154
***********************************************
test loss 0.427154
***********************************************
test loss 0.427154
***********************************************
